In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numpy import linalg
from numba import njit

On considére dans ce cas l'équation de transport lineaire avec diffusion non linéaire suivante:
\begin{equation}
    \begin{cases}
    & \frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = \frac{\partial}{\partial x}\left( \left( \mu + \frac{\tau_y}{\sqrt{\epsilon^2 + (\partial_x u)^2}} \right) \frac{\partial u}{\partial x} \right)+f(x,t) \quad \text{pour} \quad t > 0, \quad x \in [l, L] \\
    & u(x,0)=u_0(x) \quad \forall x \in [l,L]
    \end{cases}
\end{equation}

<span style="color:red;">Deuxieme Cas:</span>
$$ 
u(t, x) = 
\begin{cases} 
tx & \text{if } 0 \leq x \leq 1 \\
t & \text{if } 1 \leq x \leq 3 \\
t(4 - x) & \text{if } 3 \leq x \leq 4
\end{cases}
$$
Ce qui implique:
$$ 
u_t = 
\begin{cases} 
x & \text{if } 0 \leq x \leq 1 \\
1 & \text{if } 1 \leq x \leq 3 \\
(4-x)  & \text{if } 3 \leq x \leq 4
\end{cases}
$$
$$ 
u_x = 
\begin{cases} 
t & \text{if } 0 \leq x \leq 1 \\
0 & \text{if } 1 \leq x \leq 3 \\
-t  & \text{if } 3 \leq x \leq 4
\end{cases}
$$
et 
$$ 
 \left( \mu + \frac{\tau_y}{\sqrt{\epsilon^2 + (\frac{\partial u}{\partial x})^2}} \right) \frac{\partial u}{\partial x}  = cste \Longrightarrow{} \frac{\partial}{\partial x}\left( \left( \mu + \frac{\tau_y}{\sqrt{\epsilon^2 + (\frac{\partial u}{\partial x})^2}} \right) \frac{\partial u}{\partial x} \right)=0
$$
Par suite:
$$ 
f(x,y) = u_t+au_x-\epsilon \frac{\partial }{\partial x} ( |u_x|^p u_x) = 
\begin{cases} 
x+at & \text{if } 0 \leq x \leq 1 \\
1 & \text{if } 1 \leq x \leq 3 \\
4-x - at  & \text{if } 3 \leq x \leq 4
\end{cases}
$$

In [ ]:
" **** Solutions exactes proposées **** "

@njit(cache=True)
def u_exacte1(x,t, ALPHA, beta):
    return ALPHA * np.exp(-2 * beta * t) * np.sin(2 * np.pi * x)


@njit(cache=True)
def u_exacte2(x, t, ALPHA, beta):
    if x>= 0 and x<=1:
        return t*x
    elif x>1 and x<=3:
        return t
    elif x> 3:
        return t*(4-x)
    
" **** Les termes sources associés **** "

@njit(cache=True)
def f1(x, t, a, ALPHA, beta, epsilon,  mu ,tau):
    u  = ALPHA * np.exp(-2 * beta * t) * np.sin(2 * np.pi * x)
    term1 = -2 * beta * u
    term2 = 2 * a * np.pi * ALPHA * np.exp(-2 * beta * t) * np.cos(2 * np.pi * x)
    
    term3 = 2* np.pi * ALPHA * np.exp(-2 * beta * t) * np.cos(2 * np.pi * x)
    term4 = (4*tau*term3*np.pi**2 *u)/((epsilon**2 + term3**2)**(3/2)) * term3 - 4* u * np.pi**2 *(mu + tau/(np.sqrt(epsilon**2 + term3**2)))
    return term1 + term2 + term3


@njit(cache=True)
def f2(x, t, a, ALPHA, beta , epsilon, mu ,tau):
    
    if 0 <= x <= 1:
        return x + a * t
    elif 1 < x <= 3:
        return 1
    else:
        return 4 - x - a * t
    

" **** Le flux exact **** "

@njit#(cache=True)
def F(u, a):
    # Flux exact
    return a * u


" **** Les flux numériques proposés **** "

@njit#(cache=True)
def Rusanov(ug, ud, a, dx, dt):
    # Flux numérique Rusanov
    return 0.5 * (F(ug, a) + F(ud, a)) - a * (ud - ug)

@njit#(cache=True)
def Rusanov2(ug, ud, a, dx, dt):
    # Deuxième version du flux numérique Rusanov
    return 0.5 * (F(ug, a) + F(ud, a) - a * (ud - ug))

@njit#(cache=True)
def Roe(ug, ud, a, dx, dt):
    # Flux numérique Roe
    if a >= 0:
        return F(ug, a)
    else:
        return F(ud, a)

@njit#(cache=True)
def LF(ug, ud, a, dx, dt):
    # Flux numérique Lax-Friedrichs
    return 0.5 * (F(ug, a) + F(ud, a) - (dx / dt) * (ud - ug))

@njit#(cache=True)
def LFM(ug, ud, a, dx, dt):
    # Flux numérique Lax-Friedrichs modifié
    return 0.5 * (F(ug, a) + F(ud, a) - 0.5 * (dx / dt) * (ud - ug))

@njit(cache=True)
def LW(ug, ud, a, dx, dt):
    # Flux numérique Lax-Wendroff
    return 0.5 * (F(ug, a) + F(ud, a) - (dt / dx) * (a ** 2) * (ud - ug))

@njit#(cache=True)
def VFD(ug, ud, a, dx, dt):
    # Flux numérique volume finis decentré
    if a > 0:
        return F(ug, a)
    else:
        return F(ud, a)
    
    
@njit(cache=True)
def depature_fvc_flux(dt,alpha,x,a):
    aux=1
    eps=1e-9
    x0=0.5*(x[0]+x[1])   
    x1=x0
    xmp=x0
    while(aux>eps):
        x1=xmp-dt*alpha*a
        aux=np.abs(x0-x1)
        x0=x1
    if x0>x[1]:
        x0=x[1]
    if x0<x[0]:
        x0=x[0]    
    return x0 

 
@njit(cache=True)
def fvc_flux(u,x,dt,alpha,a):
    
    Alpha = alpha
    x_car = depature_fvc_flux(dt, Alpha,x,  a)

    up   = np.interp(x_car, x, u)
    

    return a * up



@njit(cache=True)
def VFC(U0, T,  N, a, alpha, dx, CFL,X, mu , tau, Case):
    
    if Case == 1:
        f = f1
    elif Case == 2:
        f=f2
        
    Un    = U0.copy()
    Unp1  = np.zeros(N)
    
    temps = 0
    

    
    while temps < T:
        absolute_differences = np.abs((np.diff(Un)/dx)**2)
        
        Diff = np.array([mu+tau/(np.sqrt(epsilon**2+absolute_differences[i])) for i in range(N-1)])
        
        max_diff = np.max(Diff)
        
        #dt  = CFL * dx / (a**2 * alpha) * np.abs(0.5*a-1/dx * max_diff)
        #dt  = CFL /(a/dx + 2 * np.abs(max_diff)/(dx**2))
        dt1  = CFL * (dx/(np.abs(a)*np.sqrt(2*alpha)))
        dt2  = CFL * 0.5 * dx**2 / np.abs(max_diff)
        dt   = min(dt1,dt2)
        
        
        temps += min(dt,T-dt)
        
        
        for i in range(1,N-1):
            X1   = [X[i-1],X[i]]
            u1   = [Un[i-1],Un[i]]
            
            X2   = [X[i],X[i+1]]
            u2   = [Un[i],Un[i+1]]
            
            Fg  = fvc_flux(u1,X1,dt,alpha,a)
            Fd  = fvc_flux(u2,X2,dt,alpha,a)
            
            FDd = (mu+tau/(np.sqrt(epsilon**2+((Un[i+1]-Un[i])/dx)**2))) * (Un[i+1]-Un[i])/dx
            FDg = (mu+tau/(np.sqrt(epsilon**2+((Un[i]-Un[i-1])/dx)**2))) * (Un[i]-Un[i-1])/dx

            Unp1[i] = Un[i] - dt / dx * (Fd - Fg) +  dt/dx * (FDd-FDg)+dt*f(X[i], temps, a, ALPHA, beta , epsilon, mu ,tau)
            
        Unp1[0]   = 0#Unp1[1]
        Unp1[N-1] = 0#Unp1[N-2]
        
        Un[:] = Unp1[:]
    return Un


@njit(cache=True)
def Solver(U0, a, epsilon, T,  N, CFL, scheme, mu, tau,Case):
    
    if Case == 1:
        f = f1
    elif Case == 2:
        f=f2
        
    if scheme == 0:
        flux = Roe 
    elif scheme == 1:
        flux = Rusanov
    elif scheme == 2:
        flux = Rusanov2
    elif scheme == 3:
        flux = LW
    elif scheme == 4:
        flux = VFD
    
        
    Un   = U0.copy()
    Unp1 = np.zeros(N)

    temps = 0
    
    while temps < T:
        absolute_differences = np.abs((np.diff(Un)/dx)**2)
        
        Diff = np.array([mu+tau/(np.sqrt(epsilon**2+absolute_differences[i])) for i in range(N-1)])
        
        max_diff = np.max(Diff)
        
        if scheme == 0 or scheme == 2 or scheme == 4:
            dt  = CFL /(a/dx + 2 * np.abs(max_diff)/(dx**2))
            
        if scheme == 1:
            dt  = CFL/(a/dx +  np.abs(max_diff)/(dx**2))
           
        if scheme == 3:
            dt  = CFL /(a/dx + 2 * np.abs(max_diff)/(dx**2))
        
        
        dt = min(dt, T - temps)
        temps += dt
        
        for i in range(1,N-1):
            Fd = flux(Un[i],Un[i+1],a ,dx,dt)
            Fg = flux(Un[i-1],Un[i],a ,dx,dt)
            FDd = (mu+tau/(np.sqrt(epsilon**2+((Un[i+1]-Un[i])/dx)**2))) * (Un[i+1]-Un[i])/dx
            FDg = (mu+tau/(np.sqrt(epsilon**2+((Un[i]-Un[i-1])/dx)**2))) * (Un[i]-Un[i-1])/dx

            Unp1[i] = Un[i] - dt / dx * (Fd - Fg) +  dt/dx * (FDd-FDg) + dt*f(X[i], temps, a, ALPHA, beta , epsilon, mu ,tau)
            
            
            
        Unp1[0] = 0 #Unp1[1]
        Unp1[N-1] = 0 #Unp1[N-2]


        Un[:] = Unp1[:]
        

    return Un




##############################################################################################################
Case     =  1      #premier cas / 2eme cas
T        =  1/4
l        =  0

if Case == 1:
    L = 1
    u_exacte = u_exacte1
elif Case == 2 :
    L = 4
    u_exacte = u_exacte2
a        =   2
epsilon  =   1e-3
N        =   101
CFL      =   1/2
CFL2     =   1
alpha    =   1/2

beta     =  1/2
ALPHA    =  2

X        =   np.linspace(l,L,N)
U0       =   np.array([u_exacte(x, 0 , ALPHA, beta) for x in X])
tau      =   1
mu       =   1e-3

dx = (L - l) / (N - 1)




" **** Calcul des solutions approchées **** "

f = {}

fluxes = {("Roe",0):1, ("Rusanov",1):1, ("Rusanov2",2):0, ("LW",3):0,("LFM",4):0 }

for i,j in fluxes.items():  
    if j:
        f[i[0]]  =  Solver  (U0, a, epsilon, T, N, CFL, i[1], mu, tau,Case)   
    
#U_VFC = VFC( U0, T,  N, a, alpha, dx, CFL2, X, mu , tau)

#f['VFC'] = U_VFC

/tmp/ipykernel_79077/784931025.py:223: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for i in range(1,N-1):
/tmp/ipykernel_79077/784931025.py:178: NumbaWarning: Cannot cache compiled function "Solver" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @njit(cache=True)


In [ ]:
PLOT = {"Roe":1, "Rusanov":1, "Rusanov2":1, "LW":1, "LFM":1, "VFC":1}

colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'brown']


for i, (flux, Un) in enumerate(f.items()) :
    k = list(PLOT)[i]
    if PLOT[k]:
        plt.plot(X, Un, '-', label=flux, color=colors[i])
    


#plt.xlim([1, 1.75])  
#plt.ylim([6, 12]) 

Uexact = np.array([u_exacte(x, T , ALPHA, beta) for x in X])
plt.plot(X, Uexact, '-k',  label='Uex')

#plt.xlim([-0.05, 2])  
#plt.ylim([0.6, 0.9])  

plt.legend()
plt.pause(0.1)



In [34]:
Norm = {}
for i, (flux, Un) in enumerate(f.items()):
    Norm[flux]    =  np.linalg.norm(Un-Uexact,ord = 2) 
Norm


{'Roe': 7.1647950394265205, 'Rusanov': 7.561500155205397}